In [ ]:
from dotenv import load_dotenv
import os
import pygsheets
import requests
import pandas as pd
from tqdm import tqdm

load_dotenv()

True

In [4]:
service_account = pygsheets.authorize(service_account_file='spreadsheet-automator-beora-50b41d760b99.json')

sheet = service_account.open_by_url('https://docs.google.com/spreadsheets/d/1w80E5OzCps8htsS157g-_jxQr7i6S_5etax60ZVfhz4/edit?usp=sharing')

In [5]:
test = sheet.worksheet('title', 'test')

In [6]:
df = test.get_as_df()

In [7]:
df

,random_numbers
0,0.463480
1,0.625890
2,0.746916
3,0.313408
4,0.873450
5,0.524818
6,0.228017
7,0.316090
8,0.407882
9,0.581097


## TESTING Retrieve API information

In [8]:
def get_puuid(gameName, tagline, api_key):
    link = f'https://americas.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{username}/{tagline}?api_key={api_key}'

    response = requests.get(link)
    if response.status_code == 200:
        return response.json()['puuid']
    else:
        print(f"Error: {response.status_code}")
    return None

In [9]:
def account_info_by_puuid(puuid, api_key):
    link = f'https://americas.api.riotgames.com/riot/account/v1/accounts/by-puuid/{puuid}?api_key={api_key}'

    response = requests.get(link)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
    return None

In [10]:
def retrieve_match_ids(puuid, api_key):
    link = f'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start=0&count=100&api_key={api_key}'

    response = requests.get(link)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
    return None

In [11]:
# username = 'meatbballa'
username = 'magicpp'
tagline = 'NA2'

api_key = os.getenv("riot_api_key")



puuid = get_puuid(username, tagline, api_key)

account_info_by_puuid(puuid, api_key)
match_ids = retrieve_match_ids(puuid, api_key)

In [12]:
puuid

'q6YsOcfgkZY-seouJ15T-_FKfKy8wel_789v5pGDn9ibHC8tmkEti8mOa_7lm2ltxV3YWl0l-cPEfA'

In [13]:
def retrieve_match_data(match_id, api_key):
    link = f'https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'
    response = requests.get(link)

    return response.json()

game = retrieve_match_data('NA1_5356267283', api_key)


    

In [14]:
def process_match_json(match_id, api_key):
    game = retrieve_match_data(match_id, api_key)

    metadata = game['metadata']
    data_version = metadata['dataVersion']
    match_id = metadata['matchId']
    participants_puuid = metadata['participants']

    match_info = game['info']
    match_duration = match_info['gameDuration'] # 
    match_mode = match_info['gameMode']
    match_type = match_info['gameType']
    match_version = match_info['gameVersion']
    match_map_id = match_info['mapId']
    match_queue_id = match_info['queueId']
    participant_data = match_info['participants']


    main_player = participant_data[participants_puuid.index(puuid)]
    champion_level = main_player['champLevel']
    champion_experience = main_player['champExperience']
    champion_name = main_player['championName']
    champion_id = main_player['championId']
    deaths = main_player['deaths']
    kills = main_player['kills']
    assists = main_player['assists']
    gold_earned = main_player['goldEarned']
    gold_spent = main_player['goldSpent']
    individual_position = main_player['individualPosition']
    item_0 = main_player['item0']
    item_1 = main_player['item1']
    item_2 = main_player['item2']
    item_3 = main_player['item3']
    item_4 = main_player['item4']
    item_5 = main_player['item5']
    item_6 = main_player['item6']
    lane = main_player['lane']
    neutral_minions_killed = main_player['neutralMinionsKilled']
    objectives_stolen = main_player['objectivesStolen']
    participant_id = main_player['participantId']
    riot_id_game_name = main_player['riotIdGameName']
    riot_id_tagline = main_player['riotIdTagline']
    role = main_player['role']
    total_damage_dealt = main_player['totalDamageDealt']
    total_damage_dealt_to_champions = main_player['totalDamageDealtToChampions']
    total_damage_shielded_on_teammates = main_player['totalDamageShieldedOnTeammates']
    total_damage_taken = main_player['totalDamageTaken']
    totalHealsOnTeammates = main_player['totalHealsOnTeammates']
    total_minions_killed = main_player['totalMinionsKilled']
    total_time_cc_dealt = main_player['totalTimeCCDealt']
    team_id = main_player['teamId']
    team_position = main_player['teamPosition']
    turret_kills = main_player['turretKills']
    vision_score = main_player['visionScore']
    vision_wards_bought = main_player['visionWardsBoughtInGame']
    wards_placed = main_player['wardsPlaced']
    wards_killed = main_player['wardsKilled']
    win = main_player['win']

    # perks
    main_player_perks = main_player['perks']
    stats_perks = main_player_perks['statPerks']
    perk_defense = stats_perks['defense']
    perk_flex = stats_perks['flex']
    perk_offense = stats_perks['offense']

    perk_style_selection = main_player_perks['styles']

    primary_style = perk_style_selection[0]
    primary_style_selections = primary_style['selections']
    keystone = primary_style_selections[0]
    primary_style_selection_1 = primary_style_selections[1]
    primary_style_selection_2 = primary_style_selections[2]
    primary_style_selection_3 = primary_style_selections[3]

    secondary_style = perk_style_selection[1]
    secondary_style_selections = secondary_style['selections']
    secondary_style_selection_1 = secondary_style_selections[0]
    secondary_style_selection_2 = secondary_style_selections[1]


    for team in match_info['teams']:
        team_id = team['teamId']
        team_win = team['win']
        bans = team['bans']

        if bans:
            for ban in bans:
                champion_id = ban['championId']
                # champion_name = ban['championName']


        objectives = team['objectives']
        if 'atakhan' in objectives:
            atakhan = objectives['atakhan']
        else:
            atakhan = 0
        dragon = objectives['dragon']
        inhibitor = objectives['inhibitor']
        rift_herald = objectives['riftHerald']
        tower = objectives['tower']
        baron = objectives['baron']
        horde = objectives['horde']
        champion = objectives['champion']


    match_dataframe = pd.DataFrame(
        {
            'data_version': [data_version],
            'match_id': [match_id],
            'participants_puuid': [participants_puuid],
            'match_mode': [match_mode],
            'match_type': [match_type],
            'match_duration': [match_duration],
            'match_version': [match_version],
            'match_map_id': [match_map_id],
            'match_queue_id': [match_queue_id],
            'champion_level': [champion_level],
            'champion_experience': [champion_experience],
            'champion_name': [champion_name],
            'champion_id': [champion_id],
            'deaths': [deaths],
            'kills': [kills],
            'assists': [assists],
            'gold_earned': [gold_earned],
            'gold_spent': [gold_spent],
            'individual_position': [individual_position],
            'item_0': [item_0],
            'item_1': [item_1],
            'item_2': [item_2],
            'item_3': [item_3],
            'item_4': [item_4],
            'item_5': [item_5],
            'item_6': [item_6],
            'lane': [lane],
            'neutral_minions_killed': [neutral_minions_killed],
            'objectives_stolen': [objectives_stolen],
            'participant_id': [participant_id],
            'riot_id_game_name': [riot_id_game_name],
            'riot_id_tagline': [riot_id_tagline],
            'role': [role],
            'total_damage_dealt': [total_damage_dealt],
            'total_damage_dealt_to_champions': [total_damage_dealt_to_champions],
            'total_damage_shielded_on_teammates': [total_damage_shielded_on_teammates],
            'total_damage_taken': [total_damage_taken],
            'totalHealsOnTeammates': [totalHealsOnTeammates],
            'total_minions_killed': [total_minions_killed],
            'total_time_cc_dealt': [total_time_cc_dealt],
            'team_id': [team_id],
            'team_position': [team_position],
            'turret_kills': [turret_kills],
            'vision_score': [vision_score],
            'vision_wards_bought': [vision_wards_bought],
            'wards_placed': [wards_placed],
            'wards_killed': [wards_killed],
            'win': [win],
            'keystone': [keystone],
            'primary_style_selection_1': [primary_style_selection_1],
            'primary_style_selection_2': [primary_style_selection_2],
            'primary_style_selection_3': [primary_style_selection_3],
            'secondary_style_selection_1': [secondary_style_selection_1],
            'secondary_style_selection_2': [secondary_style_selection_2],
            'team_id': [team_id],
            'team_win': [team_win],
            'atakhan': [atakhan],
            'dragon': [dragon],
            'inhibitor': [inhibitor],
            'rift_herald': [rift_herald],
            'tower': [tower],
            'baron': [baron],
            'horde': [horde],
            'champion': [champion],
        }
    )
    return match_dataframe


In [ ]:
dataframe_list = []

for match_id in tqdm(match_ids):
    match_dataframe = process_match_json(match_id, api_key)
    dataframe_list.append(match_dataframe)



 92%|█████████▏| 92/100 [00:25<00:02,  3.67it/s]


KeyError: 'metadata'

In [18]:
all_matches_dataframe = pd.concat(dataframe_list)

In [ ]:
mains = all_matches_dataframe['champion_name'].value_counts()[:4].reset_index()['champion_name'].tolist()

['Vladimir', 'Yasuo', 'Irelia', 'Yone']

In [74]:

all_matches_dataframe['champion_name'].value_counts()

champion_name
Vladimir    31
Yasuo       24
Irelia      11
Yone        11
Viego        3
Fiora        2
Camille      2
Riven        2
Olaf         1
Talon        1
Katarina     1
Sylas        1
Volibear     1
Trundle      1
Name: count, dtype: int64

In [55]:
mains_only = all_matches_dataframe[all_matches_dataframe['champion_name'].isin(mains)]

In [75]:
mains_only.groupby('champion_name')[['champion_level','turret_kills', 'total_damage_dealt_to_champions', 'wards_placed', 'gold_earned', 'total_minions_killed', 'kills', 'deaths','assists', 'win']].mean()

,champion_level,turret_kills,total_damage_dealt_to_champions,wards_placed,gold_earned,total_minions_killed,kills,deaths,assists,win
champion_name,,,,,,,,,,
Irelia,16.000000,2.818182,28352.454545,8.727273,14385.272727,227.545455,7.363636,9.272727,3.909091,0.636364
Vladimir,14.225806,1.935484,30707.161290,6.322581,11665.193548,166.870968,6.677419,8.354839,4.548387,0.451613
Yasuo,16.041667,2.083333,29174.666667,7.208333,13148.291667,209.583333,5.666667,10.125000,4.291667,0.416667
Yone,14.727273,2.000000,27478.181818,6.727273,12666.272727,177.818182,6.818182,9.454545,4.090909,0.454545


In [ ]:
vlad_matches = all_matches_dataframe[(all_matches_dataframe['champion_name'] == 'Vladimir') & (all_matches_dataframe['lane'] == 'MIDDLE')]
vlad_matches['total_damage_dealt_to_champions'].mean()
# vlad_matches['total_damage_dealt_to_champions'].median()

# how do we go about storing all the data in a db
# how do I get all the games michael has played

np.float64(31241.2)

## TESTING retrieving challenger leaderboard

In [12]:
test = requests.get(f'https://na1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key={api_key}').json()
df = pd.DataFrame(test['entries'])
df

,puuid,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,xwvxoeFlnKIvzFbEXoSPgHcsdfUBq-Nx9ZLnuBLfPmzE4A...,1978,I,477,350,True,False,False,False
1,lq-a4GXNvWCMF3ndRj5ZmV5WO_5zM2hHkPoIpQ1Aijb3Sb...,1905,I,495,409,True,False,False,False
2,1Kgk8CQHzJIpKnymtjWM0tS45nxnHZ_4wyVtl1GQAIH8-L...,1701,I,234,140,True,False,False,False
3,v0TBOhPysgl_YZfRFgHxnmfYhSs5pZnaZdXzC0fIbM-2P5...,1650,I,268,168,True,False,False,False
4,CsHA0UprPfssgTbgb92BcrDTvosGQ-0qAS1P_Nm-EiHrwi...,1630,I,294,237,True,False,False,False
...,...,...,...,...,...,...,...,...,...
295,k8WafxuFKlD1QFBDV7-Rajlf-Vhu7sTqpK9boaYTcfgYX1...,848,I,110,77,False,False,False,False
296,-HEvw0V1ZON8cS_QZyVo5IL_tlYwoAc7OeDsAD6K9QML4b...,841,I,146,76,False,False,False,False
297,Kg_sGksIHDE7KfjjvH7alv7sTYQgcZytjYiK3LPvoifBM0...,826,I,166,137,False,False,False,False
298,JZr3fNs3cdPIzu3oGwvZkUG3eagfv93Fz9Xf2oZdRZxH1u...,824,I,602,572,False,False,False,False
